In [2]:
from bs4 import BeautifulSoup

In [3]:
data:BeautifulSoup = None
with open("../data/1996.html", "r") as r:
    data = BeautifulSoup(r, "lxml")

In [4]:
from cleaner import remove_tags

resulting_soup = remove_tags(data, tags=['script', 'link', 'style','noscript'])

original = data.prettify()
modified = resulting_soup.prettify()

In [5]:
with open("../data/original.html", "w") as w:
    w.write(original)

with open("../data/modified.html", "w") as w:
    w.write(modified)